### Retrieval Augmented Generation

RAG, short for Retrieval Augmented Generation, is a natural language processing model that combines retrieval and generation techniques. Its purpose is to improve the generation of text-based responses by incorporating relevant information from external sources.

Firstly, RAG has a retrieval component that allows it to retrieve relevant passages or documents related to a given query or context. This ensures that the generated response is informed by the most up-to-date and pertinent information available. Various sources like books, articles, or the internet can be used for retrieval.

Once the relevant information is retrieved, RAG utilizes a generation component to produce coherent and contextually appropriate responses. By incorporating external knowledge, the generated text becomes more accurate, informative, and well-structured.

RAG has widespread applications, such as enhancing dialogue systems, question-answering models, and chatbots. By utilizing RAG, these systems become more reliable and effective in providing helpful responses to users' queries and needs. Overall, RAG combines retrieval and generation techniques to enhance the quality and relevance of generated text, making it an invaluable tool in natural language processing tasks.

### Notebook Details
The following Python Notebook will guide you through using the RAG APIs.

#### Assumptions/Prerequisites
- You have your own Azure storage account with the data you want to process uploaded and ready.
- The files are uploaded to a directory (not the container root).
- You have the connection string for the storage account.
- Your storage is accessible from both your local KPMG Laptop and the API compute in the Core VNET.
- You have an API key with access to the RAG product in APIM.
- You have an API key with access to the Azure OpenAI Completions/Embeddings API in APIM.
- You have a user configuration file that specifies that you have access to the "text-embedding-3-large" with an adequate quota to where you will not be throttled by the embeddings API


#### Steps
- Create a Catalog (a logical container for the data you want to index).
- Create a SAS URI that points to the data source(s) you want to process.
- Associate the SAS URI with the Catalog by creating a Catalog Source.
- Create an Ingestion Job:
  - Copy the files to a working directory inside the API private storage.
  - Extract the markdown content from all the copied files.
  - Chunk the markdown into smaller pieces.
  - Generate vectors or embeddings of the chunked contents.
  - Index a JSON representation of the chunks and vectors.
- Query the index to retrieve relevant documents.

After completing these steps, the relevant document chunks' contents can be passed to an LLM (Language Model) of your choice for summarization and analysis.

### Install the required packages

In [ ]:
#%pip install python-dotenv
#%pip install requests
#%pip install azure-storage-blob
#%pip install tabulate

### Load the configuration settings

In [ ]:
from dotenv import dotenv_values, load_dotenv
import sys
sys.path.append('../common')
from helper import Helper

load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
local_env = secrets.get('LOCAL_URL') or secrets.get("APIM_NAME")
environment = secrets.get("APIM_NAME")
account_connection_string = secrets.get("AZURE_STORAGE_CONNECTION_STRING")
region_override = secrets.get("REGION_OVERRIDE")

ragHelper = Helper(local_env, "rag/kpmg", api_key, region_override)
deploymentsHelper = Helper(environment, "genai/azure/deployments", api_key, region_override)

In [ ]:
# clean up any previous runs
response = ragHelper.get("/catalog")

assert response.status_code == 200

if (len(response.json["value"])):
    for catalog in response.json["value"]:
        response = ragHelper.delete(f"/catalog/{catalog['catalogId']}")
        assert response.status_code == 200

### Catalog Tests

##### Create a new catalog

In [ ]:
import datetime

catalogName = f'Regression Catalog: {datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")}'

ragHelper.log("Catalog Name: " + catalogName + '\n')

response = ragHelper.post("/catalog", {
    "displayName": catalogName
})

assert response.status_code == 200

catalogId = response.json["catalogId"]

ragHelper.log("Created Catalog Id: " + catalogId)

##### Attempt to create a new catalog with empty catalog name

In [ ]:
catalogName = f''

ragHelper.log("Catalog Name: " + catalogName + '\n')

response = ragHelper.post("/catalog", {
    "displayName": catalogName
})

assert response.status_code == 400

##### Generate a SAS URI for Empty catalog source

In [ ]:
from azure.storage.blob import BlobServiceClient, generate_container_sas, ContainerSasPermissions
from datetime import datetime, timedelta, timezone

source_container_name = 'reg-source'
directory_name = 'emptySourceData'

blob_service_client = BlobServiceClient.from_connection_string(account_connection_string)
account_name = blob_service_client.account_name

source_sas_token_empty = generate_container_sas(
    account_name=account_name,
    container_name=source_container_name,
    account_key=blob_service_client.credential.account_key,
    permission=ContainerSasPermissions(read=True, list=True),
    expiry=datetime.now(timezone.utc) + timedelta(hours=24)
)

source_sas_uri_empty = f"https://{account_name}.blob.core.windows.net/{source_container_name}/{directory_name}?{source_sas_token_empty}"

ragHelper.log("Empty Source SAS URI: " + source_sas_uri_empty)

##### Generate an Expired Source SAS URI

In [ ]:
source_sas_token_expired = generate_container_sas(
    account_name=account_name,
    container_name=source_container_name,
    account_key=blob_service_client.credential.account_key,
    permission=ContainerSasPermissions(read=True, list=True),
    expiry=datetime.now(timezone.utc) - timedelta(hours=24)
)

source_sas_uri_expired = f"https://{account_name}.blob.core.windows.net/{source_container_name}/{directory_name}?{source_sas_token_expired}"

ragHelper.log("Expired Source SAS URI: " + source_sas_uri_expired)

##### Attempt at Creating Catalog Source for Empty folder

In [ ]:
emptyCatalogSourceName = f'Empty Regression Catalog Source'

response2 = ragHelper.post(f"/catalog/{catalogId}/source/blob", {
    "displayName": emptyCatalogSourceName,
    "uri": source_sas_uri_empty
})

assert response2.status_code == 400

message = response2.json["message"]

ragHelper.log("Catalog Source has not been created: " + message)

##### Attempt at creating Catalog Source with long name

In [ ]:
catalogSourceName = f'abcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyz' #name more than 50 chars

response2 = ragHelper.post(f"/catalog/{catalogId}/source/blob", {
    "displayName": catalogSourceName,
    "uri": source_sas_uri_empty
})

assert response2.status_code == 400
message = response2.json["message"]

ragHelper.log("Catalog Source has not been created: " + message)

##### Attempt at Creating Catalog Source with empty name

In [ ]:
emptyCatalogSourceName = f''

response2 = ragHelper.post(f"/catalog/{catalogId}/source/blob", {
    "displayName": emptyCatalogSourceName,
    "uri": source_sas_uri_empty
})

assert response2.status_code == 400

message = response2.json["message"]

ragHelper.log("Catalog Source has not been created: " + message)

##### Attempt at Creating Catalog Source from Expired SASUri

In [ ]:
expiredCatalogSourceName = f'Expired Regression Catalog Source'

response2 = ragHelper.post(f"/catalog/{catalogId}/source/blob", {
    "displayName": expiredCatalogSourceName,
    "uri": source_sas_uri_expired
})

assert response2.status_code == 400

message = response2.json["message"]

ragHelper.log("Catalog Source has not been created: " + message)

### Start an Ingestion Job with no catalog sources

In [ ]:
response = ragHelper.post(f"/catalog/{catalogId}/ingest", None)

assert response.status_code == 404
assert response.json["message"] == "The catalog sources are either not found or you are not the creator."

### Attempt Get all Catalog Source with non existing CatalogId

In [ ]:
import uuid

newcatalogId = str(uuid.uuid4())
response = ragHelper.get(f"/catalog/{newcatalogId}/source")

assert response.status_code == 404
assert response.json["message"] == "The catalog sources are either not found or you are not the creator."

### Attempt at using job Id that doesnt match the catalog

In [ ]:
ingestionJobId = str(uuid.uuid4())
response = ragHelper.get(f"/catalog/{catalogId}/ingest/{ingestionJobId}/files")

assert response.status_code == 404
assert response.json["message"] == "The specified job does not exist."